In [ ]:
"""
This script pocesses the MNIST dataset.
Original MNIST dataset:
- Author: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
- License: Creative Commons Attribution-Share Alike 3.0 (CC BY-SA 3.0)
- Source: https://yann.lecun.com/exdb/mnist/

Processed dataset is distributed under th same license (CC BY-SA 3.0).
"""

In [ ]:
import h5py
import os


# 保存先ディレクトリを設定
output_dir = './../resources/MNIST/Qunomon'
try:
    os.makedirs(output_dir, exist_ok=True)
except Exception as e:
    print(f"ディレクトリの作成に失敗しました: {e}")
    exit(1)

train_file_path="../resources/MNIST/aug_train.h5"
test_file_path="../resources/MNIST/aug_test.h5"
import torch
from torch.utils.data import DataLoader ,random_split,TensorDataset
import h5py

def load_h5_data(h5_filename,x_name,y_name):
    """
    h5形式のデータセットからデータセットを作成する
    parameter:
        h5_filename:ファイルのパス
        x_name:ファイル内にある画像データセットの名前
        y_name:ファイル内にある正解ラベルデータセットの名前
        batch_size:バッチサイズ
    return:
        dataset:データ
    
    """
    with h5py.File(h5_filename,'r') as h5_file:
        images = h5_file[x_name][:]
        labels = h5_file[y_name][:]
    images = torch.tensor(images,dtype=torch.float32)
    labels = torch.tensor(labels,dtype=torch.long)
    dataset = TensorDataset(images,labels)
    return dataset
#データセットの読み込み
train_dataset = load_h5_data(train_file_path,'train_image','train_label')
test_dataset = load_h5_data(test_file_path,'test_image','test_label')

import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
#層化抽出法でサンプルを抽出する関数
def stratified_sample(dataset, sample_size):
    labels= [label for _, label in dataset]
    strat_split = StratifiedShuffleSplit(n_splits =1, test_size=sample_size,random_state=42)
    for _, subset_idx in strat_split.split(np.zeros(len(labels)),labels):
        subset = [dataset[i] for i in subset_idx] 
    return subset
#サンプルサイズの設定
train_sample_size= 6000
test_sample_size = 1000
#サンプルの抽出
sample_train_data = stratified_sample(train_dataset,train_sample_size)
sample_test_data = stratified_sample(test_dataset,test_sample_size)


# HDF5に保存
def save_to_hdf5_qunomon(dataset, h5_filename,x_name,y_name, target_size=(28, 28)):
    file_path = os.path.join(output_dir, h5_filename)
    try:
        with h5py.File(file_path, 'w') as h5_file:
            images = h5_file.create_dataset(x_name, (len(dataset),1, *target_size), dtype='float32',compression='gzip')
            labels = h5_file.create_dataset(y_name, (len(dataset),), dtype='int64',compression='gzip')
            batch_size =100
            image_batch=[]
            label_batch =[]
            
            for i, (image, label) in enumerate(dataset):
                image_batch.append(image)
                label_batch.append(label)

                if (i+1)% batch_size ==0 or (i+1)==len(dataset):
                    images[i+1-batch_size:i+1]=np.array(image_batch)
                    labels[i+1-batch_size:i+1]=np.array(label_batch)
                    image_batch.clear()
                    label_batch.clear()
                    
        # ファイルが存在するか確認
        if os.path.exists(file_path):
            print(f"データが {file_path} に保存されました。")  # 成功メッセージ
        else:
            print(f"ファイル保存に失敗しました: {file_path}")
    except Exception as e:
        print(f"{file_path} の保存中にエラーが発生しました: {e}")

# データを保存
save_to_hdf5_qunomon(sample_train_data, 'aug_train.h5','train_image','train_label')
save_to_hdf5_qunomon(sample_test_data, 'aug_test.h5','test_image','test_label')
